In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from planet import api
from pathlib import Path
from planet.api import downloader
from getpass import getpass

In [ ]:
planet_api_key = getpass("Input your Planet API key")

In [ ]:
planet_scene_id = '20211003_161639_91_241d'
# planet_scene_id = input("Enter a Planet Scene ID")

In [ ]:
client = api.ClientV1(api_key=planet_api_key)
req = client.get_item('PSScene', 
                      planet_scene_id)
resp = req.get()
resp.keys()

In [ ]:
client = api.ClientV1(api_key=planet_api_key)
downloader = downloader.create(client)
downloader

In [ ]:
items_to_download = [resp]
asset_types = ['ortho_analytic_8b_sr']

In [ ]:
n = len(items_to_download)
resp_ac = downloader.activate(iter(items_to_download), asset_types)
resp_ac

In [ ]:
out_path = Path('local_chips')
out_path.mkdir(exist_ok=True)

In [ ]:
n = len(items_to_download)
resp_dl = downloader.download(iter(items_to_download), asset_types, str(out_path))
resp_dl

In [ ]:
downloader.stats()

In [ ]:
downloader.shutdown()